In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Works/mental_health_balanced.csv")
print(df.head())
print(df['condition'].value_counts())

    condition                                               text
0        PTSD  I am so fucking sad I dont know why Im writing...
1      Stress  When I get stressed at work I just want to hid...
2  Loneliness  why do i have to be always alone ? Im tired of...
3        ADHD  I just showered for the first time in A MONTH!...
4     Anxiety  I got a 4.0 this semester in college! I dont e...
condition
PTSD              1000
Stress            1000
Loneliness        1000
ADHD              1000
Anxiety           1000
Schizophrenia     1000
OCD               1000
Suicidal          1000
Depression        1000
Normal            1000
EatingDisorder    1000
Addiction         1000
Bipolar           1000
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['condition'])

# Save for decoding later
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_map)

{'ADHD': np.int64(0), 'Addiction': np.int64(1), 'Anxiety': np.int64(2), 'Bipolar': np.int64(3), 'Depression': np.int64(4), 'EatingDisorder': np.int64(5), 'Loneliness': np.int64(6), 'Normal': np.int64(7), 'OCD': np.int64(8), 'PTSD': np.int64(9), 'Schizophrenia': np.int64(10), 'Stress': np.int64(11), 'Suicidal': np.int64(12)}


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # Paste your token when prompted

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import torch

In [ ]:
class MentalHealthDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = MentalHealthDataset(train_encodings, train_labels)
val_dataset = MentalHealthDataset(val_encodings, val_labels)

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = len(label_encoder.classes_)

model = AutoModelForSequenceClassification.from_pretrained(
    "mental/mental-bert-base-uncased",
    num_labels=num_labels
)


config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhaneshamanorathne1210 (dhaneshamanorathne1210-icbt-campus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.499000,0.485227
2,0.103300,0.425841
3,0.001500,0.484698
4,0.001200,0.499994


TrainOutput(global_step=5200, training_loss=0.3050488649374949, metrics={'train_runtime': 4216.5114, 'train_samples_per_second': 9.866, 'train_steps_per_second': 1.233, 'total_flos': 1.09465009225728e+16, 'train_loss': 0.3050488649374949, 'epoch': 4.0})

In [ ]:
from sklearn.metrics import classification_report

# Get predictions
preds = trainer.predict(val_dataset)
pred_labels = preds.predictions.argmax(axis=1)

# Classification report
print(classification_report(val_labels, pred_labels, target_names=label_encoder.classes_))


                precision    recall  f1-score   support

          ADHD       0.97      0.88      0.92       200
     Addiction       0.95      0.95      0.95       200
       Anxiety       0.89      0.88      0.88       200
       Bipolar       0.88      0.92      0.90       200
    Depression       0.80      0.72      0.76       200
EatingDisorder       0.96      0.98      0.97       200
    Loneliness       0.89      0.94      0.91       200
        Normal       0.91      0.89      0.90       200
           OCD       0.86      0.97      0.91       200
          PTSD       0.94      0.86      0.90       200
 Schizophrenia       0.93      0.94      0.93       200
        Stress       0.97      0.86      0.92       200
      Suicidal       0.75      0.87      0.81       200

      accuracy                           0.90      2600
     macro avg       0.90      0.90      0.90      2600
  weighted avg       0.90      0.90      0.90      2600



In [ ]:
# Save model and tokenizer
model.save_pretrained("mental_bert_model")
tokenizer.save_pretrained("mental_bert_model")

('mental_bert_model/tokenizer_config.json',
 'mental_bert_model/special_tokens_map.json',
 'mental_bert_model/vocab.txt',
 'mental_bert_model/added_tokens.json',
 'mental_bert_model/tokenizer.json')

In [ ]:
# Save label encoder
import pickle
with open("mental_label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [ ]:
import shutil
shutil.move("mental_label_encoder.pkl", "mental_bert_model/mental_label_encoder.pkl")


'mental_bert_model/mental_label_encoder.pkl'

In [ ]:
!zip -r mental_bert_package.zip mental_bert_model


  adding: mental_bert_model/ (stored 0%)
  adding: mental_bert_model/vocab.txt (deflated 53%)
  adding: mental_bert_model/tokenizer.json (deflated 71%)
  adding: mental_bert_model/mental_label_encoder.pkl (deflated 21%)
  adding: mental_bert_model/config.json (deflated 59%)
  adding: mental_bert_model/special_tokens_map.json (deflated 80%)
  adding: mental_bert_model/tokenizer_config.json (deflated 75%)
  adding: mental_bert_model/model.safetensors (deflated 8%)


In [ ]:
from google.colab import files
files.download("mental_bert_package.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def predict_condition(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([predicted])[0]


In [ ]:
sample_texts = {
    "Depression": "I feel like there’s no point in anything anymore.",
    "Anxiety": "I constantly worry about everything, even the smallest things.",
    "Stress": "Deadlines are crushing me and I can’t relax at all.",
    "PTSD": "Loud noises instantly bring me back to that awful day.",
    "Suicidal": "I’ve been thinking a lot about ending everything.",
    "Loneliness": "I feel like I have no one to talk to, ever.",
    "Bipolar": "Some days I'm on top of the world, others I can’t get out of bed.",
    "OCD": "I keep washing my hands even though I know they’re clean.",
    "Schizophrenia": "I sometimes hear voices that aren’t really there.",
    "ADHD": "I can’t focus on anything for more than a few minutes.",
    "EatingDisorder": "I feel guilty every time I eat a meal.",
    "Addiction": "No matter how hard I try, I can’t stop drinking.",
    "Normal": "Just had a chill day, watched a movie and went for a walk."
}


In [ ]:
for condition, text in sample_texts.items():
    predicted = predict_condition(text)
    print(f"Input: {condition}\nPredicted: {predicted}\n---")


Input: Depression
Predicted: Depression
---
Input: Anxiety
Predicted: Anxiety
---
Input: Stress
Predicted: Bipolar
---
Input: PTSD
Predicted: PTSD
---
Input: Suicidal
Predicted: Suicidal
---
Input: Loneliness
Predicted: Loneliness
---
Input: Bipolar
Predicted: Bipolar
---
Input: OCD
Predicted: OCD
---
Input: Schizophrenia
Predicted: Schizophrenia
---
Input: ADHD
Predicted: Depression
---
Input: EatingDisorder
Predicted: EatingDisorder
---
Input: Addiction
Predicted: Addiction
---
Input: Normal
Predicted: Bipolar
---


In [ ]:
import pickle

with open("mental_label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


In [ ]:
with open("mental_label_encoder.pkl", "rb") as f:
    loaded_label_encoder = pickle.load(f)

In [ ]:
for i in range(len(loaded_label_encoder.classes_)):
    print(f"{i} => {loaded_label_encoder.inverse_transform([i])[0]}")


0 => ADHD
1 => Addiction
2 => Anxiety
3 => Bipolar
4 => Depression
5 => EatingDisorder
6 => Loneliness
7 => Normal
8 => OCD
9 => PTSD
10 => Schizophrenia
11 => Stress
12 => Suicidal


In [ ]:
for label in loaded_label_encoder.classes_:
    print(f"{label} => {loaded_label_encoder.transform([label])[0]}")


ADHD => 0
Addiction => 1
Anxiety => 2
Bipolar => 3
Depression => 4
EatingDisorder => 5
Loneliness => 6
Normal => 7
OCD => 8
PTSD => 9
Schizophrenia => 10
Stress => 11
Suicidal => 12


In [ ]:
from google.colab import files
files.download("mental_label_encoder.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>